In [41]:
import mne 
import numpy as np
import os
import re
import csv
import random
sets = ["test","train","valid"]
list_of_channels=["Fp1","F3","F7","Fp2","F4","F8","C3","C4","O2","Fz","Cz","Pz","Oz","EMG2"] 

In [ ]:
def dataSetGenerator(numberDatapointsPerSample,data,file_id,sample_id,channel,condition,outputPath,outName, downGen):
    datapoint_step = len(data)//numberDatapointsPerSample #281 is chosen since the minimum datapoints is 70250, each sample has 250 points, hence 281 new samples generated by each sample
    if downGen:
        stepTrack = datapoint_step
        outFileName = outName+str(numberDatapointsPerSample)
    else:
        stepTrack = 1
        outFileName = outName+"_"+str(numberDatapointsPerSample)
    for i in range(stepTrack):
        subData = data[i::datapoint_step][:numberDatapointsPerSample]
        data_str  = ",".join([f"{x:.6f}" for x in subData])
        data_str = [file_id,sample_id,'EP',channel, condition, len(subData),data_str]
        outFile = os.path.join(outputPath,outFileName+".txt")
        with open(outFile, 'a') as f:
            writer = csv.writer(f, dialect = "excel-tab")
            writer.writerow(data_str)

In [43]:
def pickRandomDataPoint(data, dataDecreased):
    indices = sorted(random.sample(range(len(data)), dataDecreased+1))
    return [data[i] for i in indices]

In [ ]:
#READ scarpelli edf
def edfTotxt(sample_id, edf_file, filePath, outputPath,outName):
    rawEDF = mne.io.read_raw_edf(filePath+edf_file, preload = True)
    channels_list = rawEDF.ch_names
    condition = edf_file[0]       #1 for dreaming_no_lucid, 0 for no dreaming, 2 for lucid
    if condition == '2':
        condition = '1'
    file_id = edf_file[2:][:-4]
    subject_id = re.search(r'(\d+)_', file_id).group(1)
    print(f"Analyzing file {edf_file} with file id {file_id}, condition {condition}, subject {subject_id} with channels {channels_list}")
    

    for channel in list_of_channels:
        #extract data for each channel
        channel_idx = rawEDF.ch_names.index(channel)
        data, times = rawEDF[channel_idx, :]

        #pick random data points if channel is in list_of_channels
        if channel in channels_list:
            channel_idx = rawEDF.ch_names.index(channel)
            data, times = rawEDF[channel_idx, :]
            if len(data[0]) > 70250:
                data = pickRandomDataPoint(data[0], 70250)
            else:
                data = data[0]

        #create empty data if channel is not in list_of_channels
        elif channel not in channels_list:
            data = np.zeros(70250)
        
        #desampling samples, each sample contains 250 data points
        #removed datapoints are used to generate new samples
        #total 281 new samples are generated for each sample
        
        dataSetGenerator(numberDatapointsPerSample = 250 ,
                        data = data,
                        file_id= file_id,
                        sample_id = sample_id,
                        channel = channel,
                        condition = condition,
                        outputPath =outputPath,
                        outName = outName,
                        downGen = True)

    
    


In [45]:
def edfTotxt_konkoly(sample_id, edf_file, filePath, outputPath, outName):
    list_of_channels=["Fp1","F3","F7","Fp2","F4","F8","C3","C4","O2","Fz","Cz","Pz","Oz","EMG2"] 
    rawEDF = mne.io.read_raw_edf(filePath+edf_file, preload = True)
    channels_list = rawEDF.ch_names
    condition = '2'       #1 for dreaming_no_lucid, 0 for no dreaming, 2 for lucid
    file_id = edf_file[:-4]
    subject_id = edf_file[:-4][-3:]
    print(f"Analyzing file {edf_file}, file id {file_id}, subject {subject_id} with channels {channels_list}")

    #extract data for each channel
    for channel in list_of_channels:
        #extract data for each channel
        if channel in channels_list:
            channel_idx = rawEDF.ch_names.index(channel)
            data, times = rawEDF[channel_idx, :]
            if len(data[0]) > 70250:
                data = pickRandomDataPoint(data[0], 70250)
            else:
                data = data[0]
            if channel == 'EMG':
                channel = 'EMG2'

        #create empty data if channel is not in list_of_channels
        elif channel not in channels_list:
            data = np.zeros(70250)
        
        for eachNumb in [250]:
                dataSetGenerator(numberDatapointsPerSample = eachNumb ,
                                data = data,
                                file_id= file_id,
                                sample_id = sample_id,
                                channel = channel,
                                condition = condition,
                                outputPath = outputPath,
                                outName = outName,
                                downGen = True)

In [46]:
for whichSet in sets:
    filePath = "scarpelli/no_dreaming/"+whichSet+"/"
    filePath2 = "scarpelli/dreaming_no_lucid/"+whichSet+"/"
    lis = os.listdir(filePath)
    lis2  = os.listdir(filePath2)
    #for the first few samples
    outputPath = ""
    outputFileName = whichSet+"_"
    id = 0
    for each in lis:
        edfTotxt(id,each,filePath, outputPath,outputFileName)
        id += 1

    for each in lis2:
        edfTotxt(id,each,filePath2, outputPath,outputFileName)
        id += 1

    filePathLucid = "konkoly/lucid/"+whichSet+"/"
    lis3 = os.listdir(filePathLucid)

    for each in lis3:
        edfTotxt_konkoly(id, each, filePathLucid, outputPath, outputFileName)
        id += 1

Extracting EDF parameters from /scratch-beauty/tnguyen/lucid-dream/CNN-EEG-main-edit/dataset/scarpelli/no_dreaming/test/0_3_GB_1_R_11.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 470249  =      0.000 ...  1880.996 secs...


Analyzing file 0_3_GB_1_R_11.edf with file id 3_GB_1_R_11, condition 0, subject 3 with channels ['Fp1', 'F3', 'F7', 'FC1', 'Fp2', 'F4', 'F8', 'FC2', 'FC5', 'C3', 'T7', 'CP1', 'FC6', 'C4', 'T8', 'CP2', 'CP5', 'P3', 'P7', 'CP6', 'P4', 'P8', 'O2', 'Fz', 'Cz', 'Pz', 'Oz', 'EOG1', 'EOG2', 'EMG2']
Extracting EDF parameters from /scratch-beauty/tnguyen/lucid-dream/CNN-EEG-main-edit/dataset/scarpelli/no_dreaming/test/0_1_FA_2_R_10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 455249  =      0.000 ...  1820.996 secs...
Analyzing file 0_1_FA_2_R_10.edf with file id 1_FA_2_R_10, condition 0, subject 1 with channels ['Fp1', 'F3', 'F7', 'FC1', 'Fp2', 'F4', 'F8', 'FC2', 'FC5', 'C3', 'T7', 'CP1', 'FC6', 'C4', 'T8', 'CP2', 'CP5', 'P3', 'P7', 'CP6', 'P4', 'P8', 'O2', 'Fz', 'Cz', 'Pz', 'Oz', 'EOG1', 'EOG2', 'EMG2']
Extracting EDF parameters from /scratch-beauty/tnguyen/lucid-dream/CNN-EEG-main-edit/dataset/scarpelli/dreaming_no_lucid/test/2_1_FA_

In [ ]:
print("MISSION SUCCESSFUL")